Interesting resources:
- https://ipyvolume.readthedocs.io/en/latest/
-https://github.com/bqplot/bqplot/blob/ce27e39b60f390e89f8c0b64dd562a827fa3a391/examples/Introduction.ipynb
- Grid of plots - https://ipywidgets.readthedocs.io/en/7.6.3/examples/Layout%20Templates.html#Creating-scatter-plots-using-GridspecLayout

In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [20]:
create_float_widget = lambda x: widgets.FloatSlider(
    value=1,
    min=0,
    max=1.0,
    step=0.1,
    description='{}:'.format(x),
    disabled=False
)

dimensions_widget = widgets.IntText()
print('Size of alphabet to be coded:')
display(dimensions_widget)

out = widgets.Output()
display(out)
widget_list = []

# Handle changes to the dimension size to prob distribution inputs

def prob_eventhandler(obj):
    global widget_list
    widget_list = [create_float_widget('p{}'.format(i)) for i in range(dimensions_widget.value)]
    with out:
        clear_output()
        print('Probability distribution of alphabet:')
        display(*widget_list)


dimensions_widget.observe(prob_eventhandler, names='value')

Size of alphabet to be coded:


IntText(value=0)

Output()

In [21]:
words = ['correct', 'horse', 'battery', 'staple']
items = [widgets.Button(description=w) for w in words]
left_box = widgets.VBox([items[0], items[1]])
right_box = widgets.VBox([items[2], items[3]])
widgets.HBox([left_box, right_box])

In [22]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [31]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))

header = widgets.HTML("<h1>Infer App</h1>", layout=widgets.Layout(height='auto'))
header.style.text_align='center'

heatmap_selector = widgets.Dropdown(options=('Temperature', 'Precipitation'),
                            layout=widgets.Layout(width='auto'))

left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

In [32]:
widgets.AppLayout(header=header,
          left_sidebar=widgets.VBox([heatmap_selector]),
          center=center_button,
          right_sidebar=right_button,
          footer=footer_button)

AppLayout(children=(HTML(value='<h1>Infer App</h1>', layout=Layout(grid_area='header', height='auto')), Button…

In [25]:
# Let's begin by importing some libraries we'll need
import numpy as np
from bqplot import pyplot as plt


# And creating some random data
size = 100
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)

plt.figure(title="My First Plot")
plt.plot(x_data, y_data)
plt.show()


In [48]:
tab_contents = ['P0', 'P1']
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = ('An accordion', 'Copy of the accordion')
display(tab)



In [169]:
import io
import pandas as pd
from IPython.display import display, clear_output

uploader = widgets.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(widgets.HTML("<h1>Connect source dataframe or stream</h1>"))
display(uploader)
main_display = widgets.Output()
display(main_display)
summary_display = widgets.Output()
display(summary_display)
feature_hypos_display = widgets.Output()
display(feature_hypos_display)


connect_btn = widgets.Button(description='Connect source data', button_style='success')
view_data_btn = widgets.Button(description='View Data Viz')
next_btn = widgets.Button(description='Next', \
                                               button_style='success')
create_hyp_dropdown = lambda x: widgets.Dropdown(
    options=['Normal', 'Bernoulli', 'Poisson'],
    value='Normal',
    description='<b>Feature hypothesis:</b>',
    style= {'description_width': 'initial'}
)


global df

def show_it(inputs):
    with main_display:
        main_display.clear_output()
        file_name = list(inputs['new'].keys())[-1]
        display(widgets.HBox([widgets.HTML("<body>{}</body>".format(file_name)),connect_btn]))


        
uploader.observe(show_it, names='value')

def connect_btn_eventhandler(obj):    
    uploaded_file = uploader.value
    df = pd.read_csv(io.BytesIO(uploaded_file[file_name]['content']))
    with summary_display:
        summary_display.clear_output()
        display(widgets.HTML('<body>Connected to data source... \n</body>'))
        display(widgets.HTML(df.describe().to_html()))
        header = widgets.HTML("<h1>Source dataframe summary</h1>")
        box_layout = widgets.Layout(display='flex',
                                    flex_flow='row',
                                    # align_items='stretch', 
                                    width='50%')

        center = widgets.VBox((widgets.HTML(df.describe().to_html()),\
                               widgets.HBox((view_data_btn,\
                                           next_btn), layout=box_layout)))

        display(widgets.AppLayout(header=header,
                  # left_sidebar=widgets.VBox([heatmap_selector]),
                  center=center))

def next_btn_eventhandler(obj):
    columns = np.array(df.dtypes.index).reshape(len(df.columns),1)
    types = df.dtypes.values.reshape(len(df.columns),1)
    cols_and_types = np.concatenate((columns, types), axis=1)
    print(cols_and_types)
    with feature_hypos_display:
        feature_hypos_display.clear_output()
        output = widgets.VBox([widgets.HBox([widgets.HTML('<body><b>Column:</b> {}</body>'.format(col_ty[0])), \
                                             widgets.HTML('<body><b>Type:</b> {}</body>'.format(col_ty[1])), \
                                               create_hyp_dropdown('')]) for col_ty in cols_and_types])
        display(widgets.HTML("<h1>Choose feature hypotheses</h1>"))
        display(output)



def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

# w.observe(on_change)

# display(w)
        
connect_btn.on_click(connect_btn_eventhandler)
next_btn.on_click(next_btn_eventhandler)




HTML(value='<h1>Connect source dataframe or stream</h1>')

FileUpload(value={}, accept='.csv', description='Upload')

Output()

Output()

Output()